### 서울시 하루평균 생활인구 top5 행정동 치킨집 리뷰 데이터 크롤링(카카오맵)

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time


# 서울시 생활 인구수가 가장 많은 상위 5개 행정동 리스트 추출
data_raw = pd.read_excel('./data/서울시_행정동별_생활인구수.xlsx')
data = data_raw
data = data.sort_values(by = '총생활인구수', ascending = False)
data_top5 = data.head(5)
keywords = []
keywords = data_top5['행정동명']

# 상위 10개 각 행정동의 103개 치킨 업체(정확도 순) 리뷰 데이터 크롤링 후 csv 저장
for keyword in keywords:
    link = pd.read_csv('./data/'+keyword+" 치킨집 카카오맵 데이터 크롤링.csv")
    page_urls = []
    page_urls = link['링크주소']
    columns = ['score', 'review']
    
    # Chrome 웹드라이버 설치 경로
    excutable_path = './data/chromedriver.exe'
    driver = webdriver.Chrome(executable_path=excutable_path)
    source_url = "https://map.kakao.com/"
    # 데이터프레임 생성
    df = pd.DataFrame(columns=columns)
    
    print(keyword, '치킨집(카카오맵) 리뷰 데이터 수집 시작')
    print()

    for page_url in page_urls: # url 갯수 만큼(103개) 반복

        # 각 업체 url 접속
        driver.get(page_url)
        time.sleep(2)

        # 첫 페이지 추출
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.find(name="div", attrs={"class":"evaluation_review"})

        # 별점 추출
        rates = contents.find_all(name="em", attrs={"class":"num_rate"})

        # 리뷰 추출
        reviews = contents.find_all(name="p", attrs={"class":"txt_comment"})
        
        # 별점, 리뷰 리스트 값 한개씩 붙혀서 row(series)로 구성 후 
        # 데이터 프레임에 한줄씩 붙힘
        for rate, review in zip(rates, reviews):
            row = [rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=df.columns)
            df = df.append(series, ignore_index=True)

        # 2~5 페이지 까지 진행
        for button_num in range(2, 6):
            # 다음 페이지 버튼이 없는경우 try에서 빠져 나가도록
            try:
                more_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                more_reviews.click()
                time.sleep(2)

                # 각 페이지 추출
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents = soup.find(name="div", attrs={"class":"evaluation_review"})

                # 별점 추출
                rates = contents.find_all(name="em", attrs={"class":"num_rate"})

                # 리뷰 추출
                reviews = contents.find_all(name="p", attrs={"class":"txt_comment"})
                
                # 별점, 리뷰 리스트 값 한개씩 붙혀서 row(series)로 구성 후 
                # 데이터 프레임에 한줄씩 붙힘
                for rate, review in zip(rates, reviews):
                    row = [rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=df.columns)
                    df = df.append(series, ignore_index=True)
            except:
                break    
                
    savepath = "./data/"
    df.to_csv(savepath+keyword+" 치킨집 카카오맵 리뷰 데이터 크롤링.csv", index = False, mode='w', encoding = 'utf-8-sig')
    print(keyword+' 완료'+'================')
    print()
    driver.close()

역삼1동 치킨집(카카오맵) 리뷰 데이터 수집 시작



KeyboardInterrupt: 

In [2]:
import pandas as pd

dum = pd.read_csv('./data/역삼1동 치킨집 카카오맵 리뷰 데이터 크롤링.csv')
dum

,score,review
0,1,와....맛은 있었는데..공기밥 추가했는데 벌레나왔어요
1,5,다시 가게 될 장소. 숫불치킨 정말 맛있음
2,1,위생최악 ㅋㅋ
3,1,2명이라니까 테이블 비어있는거 뻔히 보이는데 15분 기다리라 그러고 받기 싫은 티 ...
4,3,순살먹는데 닭이 살짝 냄새나요
...,...,...
389,5,최고의 숯불바베큐 맛집!
390,1,양도적고 양념맛은 더없고..
391,4,NaN
392,4,치킨집. 서비스로 계란말이를 준다.
